<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/datasets/dataset_intro_by_doing__op_parliament.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source. 
#    (this should soon simplify to something like   !pip3 install --upgrade wetsuite)
# For your own setup, see wetsuite's install guidelines.
!pip3 install -U wetsuite

## Purpose of this notebook

Explore what is in the `parliament-sample-xml` dataset

In [32]:
import collections, random, pprint, re

import wetsuite.datasets
import wetsuite.helpers.koop_parse
import wetsuite.helpers.etree

In [5]:
px = wetsuite.datasets.load('parliament-sample-xml')

print( px.description )


                       
A moderate-sized collection of kamerstukken (~700MB of the most recently touched dossiers), handelingen (up to five years), and aanhangsels (up to five years)
                       
For each conceptual item, e.g. kst-27625-351, it contains
- a metadata XML, e.g. https://repository.overheid.nl/frbr/officielepublicaties/kst/27625/kst-27625-351/1/metadata/metadata.xml
- an content XML, e.g. https://repository.overheid.nl/frbr/officielepublicaties/kst/27625/kst-27625-351/1/xml/kst-27625-351.xml                       

The metadata file can be parsed with wetsuite.helpers.koop_parse.parse_op_metafile()

TODO: more elaboration
                                              
Dataset generated on:                        
This dataset was generated on 2024-07-23


...okay, that description needs more work.


This dataset is rawer than various others, 
to help illustrate that you could do this yourself,
with a little help from functions we wrote.

In [12]:
# For the most basic of ideas:
px.data.random_keys(5)

['https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20192020/ah-tk-20192020-1846/1/metadata/metadata.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/h-tk/20232024/h-tk-20232024-50-30/1/xml/h-tk-20232024-50-30.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/ah-tk/20202021/ah-tk-20202021-3083/1/xml/ah-tk-20202021-3083.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/kst/32637/kst-32637-242/1/xml/kst-32637-242.xml',
 'https://repository.overheid.nl/frbr/officielepublicaties/kst/30420/kst-30420-232/1/metadata/metadata.xml']

In [16]:
count = collections.defaultdict(int) 

for source_url in px.data.keys():
    if '/kst' in source_url:
        count['kst'] += 1
    elif '/h-' in source_url:
        count['h'] += 1
    elif '/ah-' in source_url:
        count['ah'] += 1
    else:
        raise ValueError("we should not have other cases")
        
count

defaultdict(int, {'ah': 37472, 'h': 28072, 'kst': 66144})

Goal: 
- First group by type of document ('ah-ek', 'ah-tk', 'h-ek', 'h-tk', 'h-vv', 'kst') 
- then group by identifier like `kst-32637-242` - because each should have exactly one metadata and one data file

For example, we might make a dict where

        {'ah-ek': 
            {'ah-ek-20182019-2': 
                {'metadata': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20182019/ah-ek-20182019-2/1/metadata/metadata.xml',
                      'xml': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20182019/ah-ek-20182019-2/1/xml/ah-ek-20182019-2.xml'
        ...   


In [21]:
def op_repo_url_parse(url):
    ''' Given a url like "https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20182019/ah-ek-20182019-2/1/metadata/metadata.xml",
        return a dict with keys like 'area' (here 'ah-ek'), 'id' (here 'ah-ek-20182019-2'), mtype (here 'metadata')
    '''
    # the regexp is ugly, and not very clear about how it was made and why it may or may not be valid - we may want something more transparent
    _re_repourl_parl = re.compile( r'^https?://repository.overheid.nl/frbr/officielepublicaties/([a-z-]+)/([A-Za-z0-9-]+)/([A-Za-z0-9_-]+)/([a-z0-9-]+)/([a-z]+)/([A-Za-z0-9_-]+.[a-z]+)$' )
    #                                                            e.g.                              h-ek       20092010     h-ek-20092010-1_2       1       xml    h-ek-20092010-1_2.xml
    m = _re_repourl_parl.match(url)
    if m is None:
        raise ValueError(f'Does not seem like a valid KOOP SRU Officiele Publicaties Repo URL: {repr(url)}')
    else:
        ret = {}
        area, _, id, mnum, mtype, basename = m.groups()
        ret['area']     = area
        ret['id']       = id
        ret['mtype']    = mtype
        ret['mnum']     = mnum
        ret['basename'] = basename
        return ret

In [23]:
groups = collections.defaultdict( lambda: collections.defaultdict(dict) ) 

for source_url in px.data.keys():
    dd = op_repo_url_parse( source_url )
    area  = dd['area']
    id    = dd['id']
    mtype = dd['mtype']

    groups[area][id][mtype] = source_url

    # if '/kst' in source_url:
    #     groups['kst'] += 1
    # elif '/h-' in source_url:
    #     count['h'] += 1
    # elif '/ah-' in source_url:
    #     count['ah'] += 1
    # else:
    #     raise ValueError("we should not have other cases")
        
dict(groups)

{'ah-ek': defaultdict(dict,
             {'ah-ek-20182019-2': {'metadata': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20182019/ah-ek-20182019-2/1/metadata/metadata.xml',
               'xml': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20182019/ah-ek-20182019-2/1/xml/ah-ek-20182019-2.xml'},
              'ah-ek-20192020-1': {'metadata': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20192020/ah-ek-20192020-1/1/metadata/metadata.xml',
               'xml': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20192020/ah-ek-20192020-1/1/xml/ah-ek-20192020-1.xml'},
              'ah-ek-20192020-10': {'metadata': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20192020/ah-ek-20192020-10/1/metadata/metadata.xml',
               'xml': 'https://repository.overheid.nl/frbr/officielepublicaties/ah-ek/20192020/ah-ek-20192020-10/1/xml/ah-ek-20192020-10.xml'},
              'ah-ek-20192020-11': {'metadata': 'http

In [64]:
import wetsuite.helpers.split
from importlib import reload
reload( wetsuite.helpers.split )

for area in groups:
    print( f'==== {area} ====' )
    for id, twofiles in groups[area].items():
        print( f'--  {id:10s} --' )
        for mtype, source_url in twofiles.items():
            if mtype == 'metadata':
                 metabytes = px.data.get(source_url)
                 display( wetsuite.helpers.koop_parse.parse_op_metafile( metabytes,as_dict=True ) )
            elif mtype == 'xml':
                 xmlbytes = px.data.get(source_url)

                 
                 #While you probably want to handle the XML contents in mor3e detail
                 #   (...uncomment the next line to get spammed with rawish XML...)
                 #display( wetsuite.helpers.etree.debug_color( xmlbytes ) )
                 #...that is a bit daunting at first, so we have some code that tries to extract just the main body of text.
                 #   (the `feeling_lucky()` call simplifies this, and this works because it has already been told 
                 #    about these document formats - you may or may not wish to learn how that actually works)
                 textary = wetsuite.helpers.split.feeling_lucky( xmlbytes ) # ()
                 print( '\n\n'.join( textary ) )

        break # print just one example from each area

==== ah-ek ====
--  ah-ek-20182019-2 --


{'documentStatus': [('', 'Opgemaakt na onopgemaakt')],
 'title': [('',
   'Vragen van het lid Van Hattem (PVV) op 2 juli 2019 medegedeeld aan de Minister van Binnenlandse Zaken en Koninkrijksrelaties inzake de brief van deze Minister aan gemeenten over verwerping door de Eerste Kamer van het wetsvoorstel verruiming ontheffing woonplaatsvereiste wethouders en gedeputeerden (34 807).')],
 'aanhangselNummer': [('', '2')],
 'indiener': [('', 'Van Hattem')],
 'ontvanger': [('', 'Ollongren')],
 'publicationName': [('', 'Kamervragen (Aanhangsel)')],
 'vergaderjaar': [('', '2018-2019')],
 'vraagnummer': [('', '218190001')],
 'language': [('DCTERMS.RFC4646', 'nl')],
 'available': [('DCTERMS.W3CDTF', '2019-09-03')],
 'issued': [('DCTERMS.W3CDTF', '2019-08-19')],
 'datumOntvangst': [('DCTERMS.W3CDTF', '2019-08-19')],
 'type': [('OVERHEID.Informatietype', 'officiële publicatie'),
  ('OVERHEIDop.AanhangselTypen', 'Antwoord'),
  ('OVERHEIDop.Parlementair', 'Aanhangsel van de Handelingen')],
 'organi

  informeert de Minister van Binnenlandse Zaken en Koninkrijksrelaties de gemeenten over de verwerping van het wetsvoorstel verruiming ontheffing woonplaatsvereiste wethouders en gedeputeerden door de Eerste Kamer. In deze brief geeft zij tevens aan dat het huidige wettelijke kader van kracht blijft. 1 Brief Ministerie van Binnenlandse Zaken en Koninkrijksrelaties, directie Democratie en Bestuur, 12 juni 2019, kenmerk: 2019-0000292304 (ter inzage gelegd op de afdeling Inhoudelijke ondersteuning onder griffienr. 165109U.JAB/fb).

 

  «Ik zal blijven streven naar meer ruimte voor lokaal maatwerk en daartoe met VNG en IPO overleg voeren, waarbij ik goede nota neem van de overwegingen van de Eerste Kamer om het wetsvoorstel te verwerpen. 

Kan de Minister aangeven wat zij concreet bedoelt met «meer ruimte voor lokaal maatwerk» en in hoeverre dit specifiek het woonplaatsvereiste, de ontheffingsmogelijkheden en het voorzitterschap van raadscommissies door niet-raadsleden betreft? 

Kan de M

/usr/local/lib/python3.8/dist-packages/wetsuite/helpers/koop_parse.py:991: UserWarning: The behaviour of alineas_with_selective_path() is not fully decided, and may still change
  warnings.warn('The behaviour of alineas_with_selective_path() is not fully decided, and may still change')


{'documentStatus': [('', 'Opgemaakt')],
 'title': [('',
   'Antwoord op vragen van het lid Sterk over de koppeling van leeftijd aan internetgames')],
 'aanhangselNummer': [('', '2635')],
 'doctype': [('', 'Officiële Publicaties, versie 1.1')],
 'indiener': [('', 'Sterk W.R.C.')],
 'publicationName': [('', 'Kamervragen (Aanhangsel)')],
 'vergaderjaar': [('', '2008-2009')],
 'vraagnummer': [('', '2009D25439')],
 'language': [('DCTERMS.RFC4646', 'nl')],
 'available': [('DCTERMS.W3CDTF', '2022-03-09')],
 'datumOntvangst': [('DCTERMS.W3CDTF', '2009-05-25')],
 'type': [('OVERHEID.Informatietype', 'officiële publicatie'),
  ('OVERHEIDop.AanhangselTypen', 'Antwoord'),
  ('OVERHEIDop.Parlementair', 'Aanhangsel van de Handelingen')],
 'organisationType': [('OVERHEID.Organisatietype', 'staten generaal')],
 'creator': [('OVERHEID.StatenGeneraal', 'Tweede Kamer der Staten-Generaal')],
 'identifier': [('OVERHEIDop.ParlID', 'ah-tk-20082009-2635')]}

...

(Zie vergadering van 14 december 1995.) 

In stemming komt de motie-Reitsma c.s. (23645, nr. 23). 

Ik constateer, dat de aanwezige leden van de fractie van de SP tegen deze
motie hebben gestemd en die van de overige fracties ervoor, zodat zij is aangenomen. 

In stemming komt de motie-Van 't Riet c.s. (23645, nr. 24). 

Ik constateer, dat deze motie met algemene stemmen is aangenomen. 

In stemming komt de motie-Remkes c.s. (23645, nr. 25). 

Ik constateer, dat de aanwezige leden van de fracties van de VVD, D66,
de RPF, de SGP, het GPV, de PvdA, het CDA, de groep-Nijpels, het AOV, de Unie
55+ en de CD voor deze motie hebben gestemd en die van de overige fracties
ertegen, zodat zij is aangenomen. 

In stemming komt de motie-M.B. Vos (23645, nr. 17). 

Ik constateer, dat de aanwezige leden van de fracties van GroenLinks,
de SP en het AOV voor deze motie hebben gestemd en die van de overige fracties
ertegen, zodat zij is verworpen. 

In stemming komt de motie-M.B. Vos (23645, nr. 18). 



{'documentStatus': [('', 'Opgemaakt')],
 'title': [('', 'Presentie en opening')],
 'eindpagina': [('', '5')],
 'handelingenItemNummer': [('', '1')],
 'publicationIssue': [('', '1')],
 'publicationName': [('', 'Handelingen')],
 'startpagina': [('', '1')],
 'vergaderjaar': [('', '2019-2020')],
 'language': [('DCTERMS.RFC4646', 'nl')],
 'available': [('DCTERMS.W3CDTF', '2019-10-09')],
 'issued': [('DCTERMS.W3CDTF', '2019-09-17')],
 'datumVergadering': [('DCTERMS.W3CDTF', '2019-09-17')],
 'type': [('OVERHEID.Informatietype', 'officiële publicatie'),
  ('OVERHEIDop.HandelingTypen', 'Opening'),
  ('OVERHEIDop.Parlementair', 'Handeling')],
 'organisationType': [('OVERHEID.Organisatietype', 'staten generaal')],
 'creator': [('OVERHEID.StatenGeneraal',
   'Verenigde Vergadering der Staten-Generaal')],
 'category': [('OVERHEID.TaxonomieBeleidsagenda', 'Bestuur | Parlement')],
 'identifier': [('OVERHEIDop.ParlID', 'h-vv-20192020-1-1')]}

  Voorzitter: Bruijn 

De  voorzitter :

De verenigde vergadering van de Staten-Generaal, als bedoeld in artikel 65 van de Grondwet, is geopend. Graag heet ik u allen van harte welkom in de Ridderzaal. 

Mededelingen 



De  voorzitter :

Ik deel aan de verenigde vergadering mee dat de volgende leden zich hebben afgemeld: 

Karabulut, Buitenweg en Gijs van Dijk. 

Deze mededeling wordt voor kennisgeving aangenomen. 

Troonrede 

Aan de orde is  het uitspreken van de troonrede .

De  voorzitter :

Ik stel aan de orde de benoeming van een Commissie van in- en uitgeleide. Zijne Majesteit de Koning zal vergezeld worden door de volgende leden van het Koninklijk Huis: Hare Majesteit Koningin Máxima, Zijne Koninklijke Hoogheid Prins Constantijn en Hare Koninklijke Hoogheid Prinses Laurentien. 

Ik benoem tot leden van de commissie die Zijne Majesteit de Koning en de overige leden van het Koninklijk Huis in- en uitgeleide zal doen: mevrouw Arib, tevens voorzitter, de heer Ganzevoort, mevrouw O

{'creator': [('OVERHEID.StatenGeneraal', 'Tweede Kamer der Staten-Generaal')],
 'identifier': [('OVERHEIDop.ParlID', 'kst-17050-229')],
 'title': [('',
   'Misbruik belastingen, sociale zekerheid en subsidies; Brief minister over de manier waarop inzicht zal worden verkregen in de omvang van de milieufraude')],
 'type': [('OVERHEID.Informatietype', 'officiële publicatie'),
  ('OVERHEIDop.Parlementair', 'Kamerstuk')],
 'available': [('DCTERMS.W3CDTF', '2002-03-22')],
 'language': [('DCTERMS.RFC4646', 'nl')],
 'category': [('OVERHEID.TaxonomieBeleidsagenda',
   'Economie | Organisatie en beleid'),
  ('OVERHEID.TaxonomieBeleidsagenda', 'Financiën | Belasting'),
  ('OVERHEID.TaxonomieBeleidsagenda',
   'Sociale zekerheid | Organisatie en beleid'),
  ('OVERHEID.TaxonomieBeleidsagenda', 'Natuur en milieu | Afval'),
  ('OVERHEID.TaxonomieBeleidsagenda',
   'Natuur en milieu | Organisatie en beleid'),
  ('OVERHEID.TaxonomieBeleidsagenda', 'Bestuur | Organisatie en beleid')],
 'organisationType

Aan de Voorzitter van de Tweede Kamer der Staten-Generaal 

's-Gravenhage,  15 maart 2002

Op 19 december 2001 is een Algemeen Overleg gevoerd met de Vaste Commissies
voor Justitie, voor Financiën, voor Sociale Zaken en Werkgelegenheid,
en voor Volksgezondheid, Welzijn en Sport over de voortgangsrapportage «Bestrijding
financieel-economische criminaliteit over het jaar 2000» (Kamerstuk
17 050, nr. 221). 

In deze voortgangsrapportage is aangegeven dat vrijwel elk milieu-onderzoek
een fraudecomponent bevat (economisch delict). Vanuit de commissies is meer
inzicht in de omvang van de milieufraude wenselijk geoordeeld. 

Mijn ambtgenoot van Justitie heeft mij gevraagd om u te berichten over
mogelijkheden om in de toekomst een nadere uitsplitsing te geven. 

Een precieze registratie is lastig, omdat het bij milieu-onderzoeken steeds
om zeer uiteenlopende misdrijven en wetsovertredingen gaat. Veelal is er geen
dan wel een indirecte relatie met eventueel door de overheid verstrekte subsidieg